In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Generate Data from A Study of Sea kayak incidents in Ireland

### Field experience

For field experienced 0 means less than 5 years, 1 means more than 5 years 2 means 10 or more years .From [1] 70% of those in the survey had been paddling  5 or more years , with nearly half of them having 10 or more experience .  Therefore I generated a data set as below.
35% - five or more experience
35% - ten or more experince 
30% less than five years experience 

In [56]:
experienced = []
for i in range(35):
    experienced.append(np.random.randint(6))
for i in range(35):
    experienced.append(np.random.randint(11))
for i in range(30):
    experienced.append(np.random.randint(20))  
np.random.shuffle(experienced)

Years_Of_Experience = pd.Series(experienced)
Years_Of_Experience

0      2
1      2
2     17
3     14
4      9
      ..
95    10
96    16
97     2
98     4
99     2
Length: 100, dtype: int64

### Field Location

For field Location 
0 means west coast
1 means east coast
2 means north coast
3 means south Coast

There was no information on location other than most incdients happened on the west coast. Thus therefore I split the 4 regions equally. 
%25 - west coast 
%25 east Coast
%25 South Coast
%25 North Coast



In [41]:

location = []
for i in range(25):
    location.append(0)
for i in range(25):
    location.append(1)
for i in range(25):
    location.append(2)
for i in range(25):
    location.append(3)    
np.random.shuffle(location)

for i in range(len(location)):
    if location[i] == 0:
        location[i] = "West Coast"
    elif location  == 1:
        location[i] = "East Coast"
    elif location  == 2:
        location[i] = "North Coast"
    else:
        location[i] = "South Coast"
location_of_incident = pd.Series(location)
location_of_incident

0     South Coast
1     South Coast
2     South Coast
3      West Coast
4      West Coast
         ...     
95    South Coast
96    South Coast
97    South Coast
98    South Coast
99    South Coast
Length: 100, dtype: object

### Field Wind Speed 

In [61]:
windSpeed = []
# 3rd of incidents happend in force 3 between 12km/h and 20 km/h
for i in range(33):
    windSpeed.append(np.random.randint(12,20))
#  80% of incidents happend in force 4 and under 
for i in range(47):
    windSpeed.append(np.random.randint(0,29))
# 20% of incidents happened above force 4 
for i in range(20):
    windSpeed.append(np.random.randint(29,40))  
np.random.shuffle(windSpeed)
Wind_Speed = pd.Series(windSpeed)
Wind_Speed

0     18
1     22
2     11
3     21
4     23
      ..
95    37
96     4
97    34
98     5
99    36
Length: 100, dtype: int64

### References 

1. Ray McCullagh and Keven 0'Callaghan,A Study of sea kayak incidents in Ireland https://issuu.com/lastudio/docs/14752_tnadt_63_issuu